In [ ]:
一. 数据集合并

问题：
合并红葡萄酒和白葡萄酒数据集（winequality-red.csv， winequality-white.csv），新增一列表示颜色，用以区分是红还是白葡萄酒
思路：
分别给红葡萄酒和白葡萄酒数据集新增一列表示颜色，再合并
主要使用的方法如下：
np.repeat()
df.rename(columns=mapping) #mapping = {a:b} 
df.append(df)

In [ ]:
# 评估数据：

import pandas as pd
import numpy as np
pd_red = pd.read_csv('winequality-red.csv', sep=';')
pd_white = pd.read_csv('winequality-white.csv', sep=';')

pd_red.info()

输出：
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
fixed_acidity           1599 non-null float64
volatile_acidity        1599 non-null float64
citric_acid             1599 non-null float64
residual_sugar          1599 non-null float64
chlorides               1599 non-null float64
free_sulfur_dioxide     1599 non-null float64
total_sulfur-dioxide    1599 non-null float64
density                 1599 non-null float64
pH                      1599 non-null float64
sulphates               1599 non-null float64
alcohol                 1599 non-null float64
quality                 1599 non-null int64
dtypes: float64(11), int64(1)
memory usage: 150.0 KB

In [ ]:
pd_white.info()

输出：
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
fixed_acidity           4898 non-null float64
volatile_acidity        4898 non-null float64
citric_acid             4898 non-null float64
residual_sugar          4898 non-null float64
chlorides               4898 non-null float64
free_sulfur_dioxide     4898 non-null float64
total_sulfur_dioxide    4898 non-null float64
density                 4898 non-null float64
pH                      4898 non-null float64
sulphates               4898 non-null float64
alcohol                 4898 non-null float64
quality                 4898 non-null int64
dtypes: float64(11), int64(1)
memory usage: 459.3 KB

In [ ]:
# 为红葡萄酒数据框创建颜色数组（其中有1599个样本）
color_red = pd.Series(np.repeat('red', 1599)) # np.repeat返回array
# 为白葡萄酒数据框创建颜色数组（其中有4898个样本）
color_white = pd.Series(np.repeat('white', 4898))

pd_red['color'] = color_red
pd_white['color'] = color_white

#查看数据框，检查是否成功
red_df.head()
white_df.head()

In [ ]:
#为合并成功，需使两个df的列名相同。将red_df中 total_sulfur-dioxide 列标签更改为 total_sulfur_dioxide
red_df = red_df.rename(columns={'total_sulfur-dioxide' : 'total_sulfur_dioxide'})

In [ ]:
#合并数据
wine_df = white_df.append(red_df)

# 查看数据框，检查是否成功
wine_df.head()
wine_df.tail()

In [ ]:
二. 数据分组聚合
红白葡萄酒问题 ：哪个水平的酸度（pH值）获得的平均评级最高？
思路：先用pd.cut() 对酸度进行划分，再用pd.groupby()对划分好的区间进行分组求评分的平均值。

In [ ]:
import pandas as pd

df = pd.read_csv('winequality_edited.csv')  #'winequality_edited.csv'这是合并后的数据

# 用 Pandas 描述功能查看最小、25%、50%、75% 和 最大 pH 值 
pd.describe()['pH']

输出：

count    6497.000000
mean        3.218501
std         0.160787
min         2.720000
25%         3.110000
50%         3.210000
75%         3.320000
max         4.010000
Name: pH, dtype: float64

In [ ]:
# 对用于把数据“分割”成组的边缘进行分组
bin_edges = [ 2.72, 3.11, 3.21, 3.32, 4.01]# 用刚才计算的五个值填充

'''

酸度水平：
高: 最低 25% 时的 pH 值
中等偏高: 25% - 50% 时的 pH 值
中: 50% - 75% 时的 pH 值
低: 最高 75% 时的 pH 值
'''

# 四个酸度水平组的标签
bin_names = [ 'high', 'medium_to_high', 'medium', 'low']# 对每个酸度水平类别进行命名

In [ ]:
# 创建 acidity_levels 列
df['acidity_levels'] = pd.cut(df['pH'], bin_edges, labels=bin_names)

# 检查该列是否成功创建
df.head()

In [ ]:
# 用 groupby 计算每个酸度水平的平均质量
df.groupby('acidity_levels')['quality'].mean()

In [ ]:
# 保存更改
df.to_csv('winequality_edited.csv', index=False)

In [ ]:
三. 通过查询进行数据分组

问题 ：酒精含量越高的葡萄酒获得的评级更高吗？
要回答这个问题，请使用 query 创建两组葡萄酒样本:
低酒精（酒精含量低于中值的样本）
高酒精（酒精含量高于或等于中值的样本）
然后，找到每组的平均质量评级。

In [ ]:
# 加载 `winequality_edited.csv`
import pandas as pd
df = pd.read_csv('winequality_edited.csv')

In [ ]:
# 获取酒精含量的中位数
med = df['alcohol'].median()

# 选择酒精含量小于中位数的样本
low_alcohol = df.query('alcohol<@med') # 等于df[df['alcohol']<med]

# 选择酒精含量大于等于中位数的样本
high_alcohol = df.query('alcohol>=@med')

In [ ]:
# 确保这些查询中的每个样本只出现一次
num_samples = df.shape[0]

num_samples == low_alcohol['quality'].count() + high_alcohol['quality'].count() # 应为真

In [ ]:
# 获取低酒精含量组和高酒精含量组的平均质量评分，并进行比较判断
low_alcohol['quality'].mean()  <  high_alcohol['quality'].mean()